## Multi-head attention transformer
### Encoder and Decoder
### (With masking)

Pytorch's implementation (in built)

NOTE :- A new exmple must be used for testing the masked attention

In [163]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return self.encoding[:, :x.size(1)].detach()

class TransformerModel(nn.Module):
    
    def __init__(self, src_vocab_size, tgt_vocab_size,max_seq_len, d_model=4, nhead=2, num_encoder_layers=1, num_decoder_layers=1):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0, max_len=max_seq_len)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=0
        )
        self.fc = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):

        print("Source embeddings :- \n")
        print(self.src_embedding(src))
        print()

        print("Target embeddings :- \n")
        print(self.tgt_embedding(tgt))
        print()


        src = self.src_embedding(src) + self.positional_encoding(src)
        tgt = self.tgt_embedding(tgt) + self.positional_encoding(tgt)

        print("Positional encoded source embeddings:")
        print(src)
        print()

        print("Positional encoded target embeddings:")
        print(tgt)
        print()

        output = self.transformer(src, tgt)
        output = self.fc(output)
        
        return output

    

src_vocab_size = 10  # Source language vocabulary size
tgt_vocab_size = 10  # Target language vocabulary size
d_model = 6  # Dimension of the model
num_heads = 2
max_seq_len = 4
model = TransformerModel(src_vocab_size, tgt_vocab_size, d_model=d_model, max_seq_len = max_seq_len, nhead=num_heads)

# Source sentence in the source language
# Source token indexes from src vocabulary
src_sentence = torch.tensor([[0], [1], [2], [3]]) 



# Target sentence in the target language (translation of the source sentence)
# Target token indexes from tgt vocabulary
tgt_sentence = torch.tensor([[1], [0], [3], [3]])

# Forward pass
output = model(src_sentence, tgt_sentence)
print(output, output.shape)



Source embeddings :- 

tensor([[[-2.2991, -1.0892, -0.6579,  1.2630, -0.6850, -2.1738]],

        [[-0.7063, -0.1283, -0.1086, -0.9614, -0.9128,  1.4580]],

        [[ 0.5709,  0.3660,  0.1946,  0.2407, -0.2572,  0.0726]],

        [[-0.1525, -0.2407,  0.3625, -0.2211, -0.9160,  0.5704]]],
       grad_fn=<EmbeddingBackward0>)

Target embeddings :- 

tensor([[[-0.0037, -0.4092,  0.6838,  0.4102, -0.8705,  0.3255]],

        [[ 0.4654, -1.9693, -0.2439,  0.3972, -0.4163, -0.5562]],

        [[-1.9848, -0.5422,  0.6171, -1.1887, -0.1505,  0.1084]],

        [[-1.9848, -0.5422,  0.6171, -1.1887, -0.1505,  0.1084]]],
       grad_fn=<EmbeddingBackward0>)

Positional encoded source embeddings:
tensor([[[-2.2991, -0.0892, -0.6579,  2.2630, -0.6850, -1.1738]],

        [[-0.7063,  0.8717, -0.1086,  0.0386, -0.9128,  2.4580]],

        [[ 0.5709,  1.3660,  0.1946,  1.2407, -0.2572,  1.0726]],

        [[-0.1525,  0.7593,  0.3625,  0.7789, -0.9160,  1.5704]]],
       grad_fn=<AddBackward0>)

Posi

## Functions to get the manual calulations for each component

In [164]:
from torch import functional as F

# Using the state dictionary to get the intermediate outputs
state_dict = model.state_dict()

need_weights = False

src_mask = None

tgt_mask = None

memory_mask = None

embed_dim = 6

num_heads = 2

max_seq_len = 4


In [165]:
def look_up_table(sentence, vocab_embeds, embedding):

    for i in range(sentence.size(0)):
        for j in range(sentence.size(1)):
            # Get the index for the current word token index in the sequence
            word_index = sentence[i, j].item()

            if word_index < 0 or word_index >= vocab_embeds.size(0):
                raise ValueError(f"Invalid word index: {word_index}")

            # Lookup the corresponding embedding vector for the word
            embedding[i, j, :] = vocab_embeds[word_index, :]

            print(f"Word index: {word_index}, Embedding: {vocab_embeds[word_index, :]}")
    print()
    

    return embedding

### Embeddings and Positional encoding

In [166]:
def get_embedding_outputs(src_sentence,  tgt_sentence, max_seq_len, state_dict, d_model):

    src_vocab_embeds = state_dict["src_embedding.weight"]

    src_embedding = torch.zeros(src_sentence.size(0), src_sentence.size(1), d_model)
    print("Source sentence embedding")
    src_embedding =  look_up_table(src_sentence, src_vocab_embeds, src_embedding)
    print(src_embedding.shape)

    tgt_vocab_embeds = state_dict["tgt_embedding.weight"]

    tgt_embedding = torch.zeros(tgt_sentence.size(0), tgt_sentence.size(1), d_model)


    print("Target sentence embedding")
    tgt_embedding =  look_up_table(tgt_sentence, tgt_vocab_embeds, tgt_embedding)

    pe = PositionalEncoding(d_model = d_model, dropout=0, max_len=max_seq_len)

    print("PE of src :")
    print(pe(src_embedding))
    print()
    print("PE of tgt :")
    print(pe(tgt_embedding))
    print()

    pe_src_embeds = src_embedding + pe(src_embedding)

    pe_tgt_embeds = tgt_embedding + pe(tgt_embedding)

    print("PE source embeddings : \n")
    print(pe_src_embeds)
    print()

    print("PE target embeddings : \n")
    print(pe_tgt_embeds)
    print()

    return pe_src_embeds, pe_tgt_embeds



In [167]:
pe_src_embeds, pe_tgt_embeds = get_embedding_outputs(src_sentence=src_sentence,  tgt_sentence=tgt_sentence, state_dict=state_dict, max_seq_len=max_seq_len, d_model = d_model)


Source sentence embedding
Word index: 0, Embedding: tensor([-2.2991, -1.0892, -0.6579,  1.2630, -0.6850, -2.1738])
Word index: 1, Embedding: tensor([-0.7063, -0.1283, -0.1086, -0.9614, -0.9128,  1.4580])
Word index: 2, Embedding: tensor([ 0.5709,  0.3660,  0.1946,  0.2407, -0.2572,  0.0726])
Word index: 3, Embedding: tensor([-0.1525, -0.2407,  0.3625, -0.2211, -0.9160,  0.5704])

torch.Size([4, 1, 6])
Target sentence embedding
Word index: 1, Embedding: tensor([-0.0037, -0.4092,  0.6838,  0.4102, -0.8705,  0.3255])
Word index: 0, Embedding: tensor([ 0.4654, -1.9693, -0.2439,  0.3972, -0.4163, -0.5562])
Word index: 3, Embedding: tensor([-1.9848, -0.5422,  0.6171, -1.1887, -0.1505,  0.1084])
Word index: 3, Embedding: tensor([-1.9848, -0.5422,  0.6171, -1.1887, -0.1505,  0.1084])

PE of src :
tensor([[[0., 1., 0., 1., 0., 1.]]])

PE of tgt :
tensor([[[0., 1., 0., 1., 0., 1.]]])

PE source embeddings : 

tensor([[[-2.2991, -0.0892, -0.6579,  2.2630, -0.6850, -1.1738]],

        [[-0.7063,  

## Encoder function to display the intermediate outputs and get the final outputs from the encoder

### Self attention 

In [168]:
def atten_product_needs_wts_false(Q, V, K, bsz, head_dim, src_len, tgt_len, attn_mask):


    # *** For multi-head attention ***
    #  (bsz*num_heads, src_len , head_dim) -> (bsz, num_heads, tgt_len, head_dim)
    Q1 = Q.view(bsz, num_heads, tgt_len, head_dim)
    K1 = K.view(bsz, num_heads, src_len, head_dim)
    V1 = V.view(bsz, num_heads, src_len, head_dim)

    L, S = Q1.size(-2), K1.size(-2)

    scale_factor = 1 / math.sqrt(Q1.size(-1)) 
    # scale_factor = 1
    attn_bias = torch.zeros(L, S, dtype=Q1.dtype)

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_mask.masked_fill_(attn_mask.logical_not(), float("-inf"))
        else:
            attn_bias += attn_mask

    # (bsz, num_heads, tgt_len, head_dim) @ (bsz, num_heads, head_dim, tgt_len) -> (bsz, num_heads, tgt_len, tgt_len) 
    attn_weight = Q1 @ K1.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias

    # (bsz, num_heads, tgt_len, tgt_len) 
    attn_weight = torch.softmax(attn_weight, dim=-1)

    print("Attention weights = ")
    print(attn_weight.shape, attn_weight)

    # (bsz, num_heads, tgt_len, tgt_len) @ (bsz, num_heads, tgt_len, head_dim) -> (bsz, num_heads, tgt_len, head_dim) 
    attn_output = attn_weight @ V1

    # (bsz*tgt_len, embed_dim)
    attn_output = attn_output.permute(2, 0, 1, 3).contiguous().view(bsz * tgt_len, embed_dim)

    return attn_output



def atten_product_needs_wts_true(Q, K, V, bsz, tgt_len, attn_mask):

    # *** For multi-head attention ***
    #  (bsz*num_heads, src_len , head_dim)
    B, Nt, E = Q.shape

    Q_scaled = Q / math.sqrt(E)

    if attn_mask is not None:
        temp_pdt_matrix = torch.baddbmm(attn_mask, Q_scaled, K.transpose(-2, -1))
    else:
        temp_pdt_matrix = torch.bmm(Q_scaled, K.transpose(-2, -1))

    attn_wt_matrix = torch.nn.functional.softmax(temp_pdt_matrix, dim=-1)

    attn_output = torch.bmm(attn_wt_matrix, V)

    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len * bsz, embed_dim)

    print("Encoder Attention output = ")
    print(attn_output)
    print()

    return attn_output, attn_wt_matrix


In [169]:
def get_qkv(query, key, value ,W, b):

    # embed_dim
    E = query.size(-1)

    if key is value:
        if query is key:
            
            # (src_len, bsz, embed_dim) @ (embed_dim*num_heads, embed_dim).T -> (src_len, bsz, embed_dim*num_heads)
            tempop1 = query@W.T

            # reshape to 3, E and not E, 3 is deliberate for better memory coalescing and keeping same order as chunk()
            tempop1 = tempop1.unflatten(-1, (3, E)).unsqueeze(0).transpose(0, -2).squeeze(-2).contiguous()

            # (src_len, bsz, embed_dim)
            return tempop1[0], tempop1[1], tempop1[2]
        

        else:

            # (embed_dim*1, embed_dim)
            # (embed_dim*2, embed_dim)
            W_q, W_kv = W.split([E, E * 2])

            if b is None:
                b_q = b_kv = None
            else:
                b_q, b_kv = b.split([E, E * 2])

            # (src_len, bsz, embed_dim) @ (embed_dim*1, embed_dim).T -> (src_len, bsz, embed_dim)
            q_matmul = query@W_q.T

            # (src_len, bsz, embed_dim) @ (embed_dim*2, embed_dim).T -> (src_len, bsz, embed_dim*2)
            kv_matmul = key@W_kv.T

            kv_matmul = kv_matmul.unflatten(-1, (2, E)).unsqueeze(0).transpose(0, -2).squeeze(-2).contiguous()

            # (src_len, bsz, embed_dim)
            return q_matmul, kv_matmul[0], kv_matmul[1]

    else:

        W_q, W_k, W_v = W.chunk(3)
        if b is None:
            b_q = b_k = b_v = None
        else:
            b_q, b_k, b_v = b.chunk(3)


        q_matmul = query@W_q.T
        k_matmul = key@W_k.T
        v_matmul = value@W_v.T

        return q_matmul, k_matmul, v_matmul



    

In [170]:
def encoder_block_attn_output(x, state_dict, layer_num, embed_dim, num_heads, need_weights = False, src_mask = None):

    # (src_len, bsz, embed_dim)
    query_enc = key_enc = value_enc = x

    tgt_len, bsz, embed_dim = x.shape

    # print(x.shape)
    
    # (embed_dim*num_heads, embed_dim)
    W_enc = state_dict["transformer.encoder.layers.{}.self_attn.in_proj_weight".format(layer_num)]
    b_enc = state_dict["transformer.encoder.layers.{}.self_attn.in_proj_bias".format(layer_num)]


    head_dim = embed_dim//num_heads
    
    # (src_len, bsz, embed_dim)
    Q_enc,K_enc,V_enc = get_qkv(query_enc, key_enc, value_enc ,W_enc, b_enc)

    # (1, src_len, bsz, embed_dim)
    Q_enc = Q_enc.unsqueeze(0)
    K_enc = K_enc.unsqueeze(0)
    V_enc = V_enc.unsqueeze(0)

    # (1, src_len, bsz, embed_dim) -> ( bsz*num_heads, src_len , head_dim)
    Q_enc = Q_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_enc = K_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_enc = V_enc.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_enc_{} = ".format(layer_num))
    print(Q_enc)
    print()

    print("K_enc_{} = ".format(layer_num))
    print(K_enc)
    print()

    print("V_enc_{} = ".format(layer_num))
    print(V_enc)
    print()


    src_len = K_enc.size(1)

    attn_mask = src_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)


    if need_weights is False:

        attn_output = atten_product_needs_wts_false(Q_enc, V_enc, K_enc, bsz, head_dim, src_len, tgt_len, attn_mask)

        return attn_output, src_len, head_dim, None
    
    else:

        attn_enc_output,attn_wt_matrix_enc = atten_product_needs_wts_true(Q_enc, K_enc, V_enc, bsz, tgt_len, attn_mask)

        return attn_enc_output, src_len, head_dim, attn_wt_matrix_enc

    

In [171]:
# need_weights = True

attn_enc_output, src_len, head_dim, attn_weights = encoder_block_attn_output(pe_src_embeds, state_dict, layer_num = 0, need_weights = need_weights, embed_dim=embed_dim, num_heads=num_heads)


Q_enc_0 = 
tensor([[[ 0.1998,  0.7083,  0.2082],
         [-0.0606, -0.1456,  0.5500],
         [ 1.0679, -0.5471,  0.5807],
         [ 0.1715, -0.0964,  0.6861]],

        [[ 0.1071,  0.2617, -0.8886],
         [ 1.3566, -0.1725,  0.0735],
         [ 0.9165, -0.0318,  0.7110],
         [ 1.1417, -0.2430,  0.4955]]])

K_enc_0 = 
tensor([[[-0.5979, -1.1891, -1.7691],
         [-1.6906, -0.9134,  0.3297],
         [-1.0899, -0.8780, -0.0898],
         [-1.3269, -0.7487, -0.0347]],

        [[ 0.6594, -0.9362,  0.5301],
         [-1.1148, -0.2258,  0.5421],
         [-0.1797, -0.4961,  0.9682],
         [-0.8286, -0.3978,  0.4431]]])

V_enc_0 = 
tensor([[[-1.6108,  1.0090,  1.4326],
         [-1.8543, -0.0398,  0.9824],
         [-0.5641,  0.2149,  0.6389],
         [-1.1238, -0.0103,  1.0536]],

        [[-0.1975, -0.3194, -1.7217],
         [-0.9378, -0.2746, -1.1484],
         [ 0.2284, -0.2286, -0.8718],
         [-0.4016, -0.0044, -0.8831]]])

Attention weights = 
torch.Size([1, 2, 4

### Post self attention in encoder

In [172]:
def encoder_block_post_attn_output(x, attn_enc_output, state_dict, layer_num, bsz, tgt_len):

    # (bsz*src_len , embed_dim) @ (embed_dim , embed_dim).T -> (bsz*src_len , embed_dim)
    op_enc_1 = torch.matmul(attn_enc_output, state_dict["transformer.encoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]



    # (bsz*src_len , embed_dim) -> (src_len, bsz, embed_dim)
    attn_enc_output = op_enc_1.view(tgt_len, bsz, attn_enc_output.size(1))


    # Here src is the original passed inputs to the 1st transformer encoder layer which 
    # are pe_src_embeds 

    # (src_len, bsz, embed_dim)
    output_enc_1 = attn_enc_output + x

    #  (src_len, bsz, embed_dim) @ (embed_dim) -> (src_len, bsz, embed_dim) 
    linear_result_enc_1 = output_enc_1*state_dict["transformer.encoder.layers.{}.norm1.weight".format(layer_num)] + state_dict["transformer.encoder.layers.{}.norm1.bias".format(layer_num)]

    # Layer normalization from Torch's implementation
    layernorm_enc_1 = torch.nn.LayerNorm(normalized_shape=linear_result_enc_1.shape[2:])
    linear_op_enc_1 = layernorm_enc_1(linear_result_enc_1)


    # Manual Layer Normalization
    x = linear_result_enc_1

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_enc_1.weight
    b = layernorm_enc_1.bias

    linear_result_enc_1f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_enc_1f.mean(dim=-1, keepdim=True)
    std = linear_result_enc_1f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_enc_1 = (linear_result_enc_1f - mean) / (std + epsilon) * w + b


    op_enc_1 = torch.matmul(normalized_result_enc_1, state_dict["transformer.encoder.layers.{}.linear1.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.linear1.bias".format(layer_num)]
    op_enc_1_relu = torch.nn.functional.relu(op_enc_1)
    op_enc_2 = torch.matmul(op_enc_1_relu, state_dict["transformer.encoder.layers.{}.linear2.weight".format(layer_num)].t()) + state_dict["transformer.encoder.layers.{}.linear2.bias".format(layer_num)]


    output_enc_2 = op_enc_2 + linear_op_enc_1
    output_enc_2_norm = output_enc_2*state_dict["transformer.encoder.layers.{}.norm2.weight".format(layer_num)] + state_dict["transformer.encoder.layers.{}.norm2.bias".format(layer_num)]

    # Layer normalization from Torch's implementation
    layernorm_enc_final = torch.nn.LayerNorm(normalized_shape=output_enc_2_norm.shape[2:])
    output_enc_final = layernorm_enc_final(output_enc_2_norm)


    # Manual Layer Normalization 
    x = output_enc_2_norm

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_enc_final.weight
    b = layernorm_enc_final.bias

    linear_result_enc_2 = w*x + b

    epsilon = 1e-5  
    mean = linear_result_enc_2.mean(dim=-1, keepdim=True)
    std = linear_result_enc_2.std(dim=-1, unbiased=False, keepdim=True)
    output_enc_final = (linear_result_enc_2 - mean) / (std + epsilon) * w + b

    print("Final Encoder {} Output :".format(layer_num))
    print("norm2(norm1(x + self_atten(x)) + feed_fwd_op)\n")
    print(output_enc_final)
    print()

    # (src_len, bsz, embed_dim) 
    return output_enc_final


In [173]:
tgt_len, bsz, embed_dim = pe_src_embeds.shape
output_enc_final = encoder_block_post_attn_output(pe_src_embeds, attn_enc_output, state_dict, layer_num = 0 , bsz = bsz, tgt_len = tgt_len)

Final Encoder 0 Output :
norm2(norm1(x + self_atten(x)) + feed_fwd_op)

tensor([[[-1.0754,  0.1729, -0.9973,  1.8929, -0.3399,  0.3467]],

        [[-0.7170,  0.0998, -0.8355,  0.6011, -0.9979,  1.8495]],

        [[-0.0678,  0.3354, -1.2914,  1.5234, -1.1904,  0.6909]],

        [[-0.4521, -0.0402, -0.7991,  1.2851, -1.3204,  1.3268]]],
       grad_fn=<AddBackward0>)



## Decoder functions to display the intermediate outputs and get the final outputs from the decoder


### Self attention outputs from a decoder block

In [174]:
def decoder_block_self_attn_output(x, state_dict, layer_num, tgt_mask = None,need_weights = False):

    # (tgt_len, bsz, embed_dim)
    query_dec = key_dec = value_dec = x

    tgt_len, bsz, embed_dim = x.shape
 
    # (embed_dim*num_heads, embed_dim)
    W_dec = state_dict["transformer.decoder.layers.{}.self_attn.in_proj_weight".format(layer_num)]
    b_dec = state_dict["transformer.decoder.layers.{}.self_attn.in_proj_bias".format(layer_num)]


    head_dim = embed_dim//num_heads

    # (tgt_len, bsz, embed_dim)
    Q_dec,K_dec,V_dec = get_qkv(query_dec, key_dec, value_dec ,W_dec, b_dec)
    
    Q_dec = Q_dec.unsqueeze(0)
    K_dec = K_dec.unsqueeze(0)
    V_dec = V_dec.unsqueeze(0)

    # (1, tgt_len, bsz, embed_dim)
    print(Q_dec.shape, K_dec.shape , V_dec.shape)
    print(tgt_len, bsz * num_heads, head_dim)

    # (1, tgt_len, bsz, embed_dim) -> ( bsz*num_heads, tgt_len , head_dim )
    Q_dec = Q_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_dec = K_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_dec = V_dec.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_dec_{} = ".format(layer_num))
    print(Q_dec)
    print()

    print("K_dec_{} = ".format(layer_num))
    print(K_dec)
    print()

    print("V_dec_{} = ".format(layer_num))
    print(V_dec)
    print()

    src_len = K_dec.size(1)


    attn_mask = tgt_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)


    

    if need_weights is False:
        attn_output = atten_product_needs_wts_false(Q_dec, V_dec, K_dec, bsz, head_dim, src_len, tgt_len, attn_mask)

        print("Decoder Self Attention = ")
        print(attn_output)
        print()

        op_dec_1 = torch.matmul(attn_output, state_dict["transformer.decoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_1.view(tgt_len, bsz, attn_output.size(1))

        return attn_dec_output, None
    
    else:

        attn_dec_output,attn_wt_matrix_dec = atten_product_needs_wts_true(Q_dec, K_dec, V_dec, bsz, tgt_len, attn_mask)

        print("Decoder Attention output = ")
        print(attn_wt_matrix_dec)
        print()

        op_dec_1 = torch.matmul(attn_dec_output, state_dict["transformer.decoder.layers.{}.self_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.self_attn.out_proj.bias".format(layer_num)]
        attn_dec_output = op_dec_1.view(tgt_len, bsz, attn_dec_output.size(1))
    

        return attn_dec_output, attn_wt_matrix_dec


    

In [175]:
need_weights 

False

In [176]:
# need_weights = True

self_attn_dec, dec_sa_wts = decoder_block_self_attn_output(pe_tgt_embeds, state_dict, layer_num = 0, need_weights = need_weights)

torch.Size([1, 4, 1, 6]) torch.Size([1, 4, 1, 6]) torch.Size([1, 4, 1, 6])
4 2 3
Q_dec_0 = 
tensor([[[-0.3784, -0.7437, -0.6235],
         [-0.7424, -0.8504, -0.4722],
         [ 0.1257,  0.3638,  0.5637],
         [ 0.1257,  0.3638,  0.5637]],

        [[-0.4801,  0.2827, -0.1775],
         [ 0.1221,  0.4419,  0.8983],
         [-0.1287, -0.7501, -1.4718],
         [-0.1287, -0.7501, -1.4718]]])

K_dec_0 = 
tensor([[[ 0.0865, -1.2798,  0.5098],
         [-0.1128, -1.0767,  0.9138],
         [ 0.9444, -0.9628, -0.7018],
         [ 0.9444, -0.9628, -0.7018]],

        [[ 0.5999,  0.1313,  0.3402],
         [ 0.4384,  0.6527,  0.9106],
         [-1.2218,  0.0632, -1.2223],
         [-1.2218,  0.0632, -1.2223]]])

V_dec_0 = 
tensor([[[-1.1569, -1.4169, -0.2169],
         [-0.1712, -0.1728, -0.3195],
         [-1.0927, -1.3739,  0.3832],
         [-1.0927, -1.3739,  0.3832]],

        [[ 0.5000, -1.5058,  0.7683],
         [ 0.0646, -0.6681,  0.5009],
         [ 0.4547,  0.2847, -0.4699],


In [177]:
self_attn_dec

tensor([[[-0.8188, -0.4678,  0.6516, -0.6265, -0.2877, -0.0952]],

        [[-0.6577, -0.3695,  0.6269, -0.2723, -0.1989, -0.1276]],

        [[-0.7676, -0.4025,  0.4873, -0.7457, -0.2433,  0.0417]],

        [[-0.7676, -0.4025,  0.4873, -0.7457, -0.2433,  0.0417]]])

In [178]:
def dec_post_self_attn(self_attn_dec, x, state_dict, layer_num):

    # (bsz*tgt_len , embed_dim) @ (embed_dim , embed_dim).T -> (bsz*tgt_len , embed_dim)
    output_dec_1 = self_attn_dec + x

    # (bsz*tgt_len , embed_dim) @ (embed_dim , embed_dim).T -> (bsz*tgt_len , embed_dim)
    linear_result_dec_1 = output_dec_1*state_dict["transformer.decoder.layers.{}.norm1.weight".format(layer_num)] + state_dict["transformer.decoder.layers.{}.norm1.bias".format(layer_num)]

    layernorm_dec_1 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_1.shape[2:])
    linear_op_dec_1 = layernorm_dec_1(linear_result_dec_1)

    # From torch's implementation
    # print(linear_op_dec_1)

    x = linear_result_dec_1

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_dec_1.weight
    b = layernorm_dec_1.bias

    linear_result_dec_1f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_1f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_1f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_1 = (linear_result_dec_1f - mean) / (std + epsilon) * w + b


    print("Decoder_{} norm1(x + sa(x))".format(layer_num))
    print(normalized_result_dec_1)
    print()

    # (tgt_len, bsz, embed_dim)
    return normalized_result_dec_1

In [179]:
x_dec = dec_post_self_attn(self_attn_dec, pe_tgt_embeds, state_dict, layer_num = 0)

Decoder_0 norm1(x + sa(x))
tensor([[[-1.1113, -0.1304,  1.1276,  0.5552, -1.4596,  1.0186]],

        [[-0.1763, -1.6359,  0.5559,  1.5003, -0.7148,  0.4709]],

        [[-1.8464,  0.2633,  1.0516, -0.4803, -0.0742,  1.0860]],

        [[-1.8464,  0.2633,  1.0516, -0.4803, -0.0742,  1.0860]]],
       grad_fn=<AddBackward0>)



## Cross attention in decoder 
### query, key, value =  x, mem, mem 

### Cross attention between the encoder's final output and the decoder layer 

In [180]:

memory = output_enc_final
x_dec

tensor([[[-1.1113, -0.1304,  1.1276,  0.5552, -1.4596,  1.0186]],

        [[-0.1763, -1.6359,  0.5559,  1.5003, -0.7148,  0.4709]],

        [[-1.8464,  0.2633,  1.0516, -0.4803, -0.0742,  1.0860]],

        [[-1.8464,  0.2633,  1.0516, -0.4803, -0.0742,  1.0860]]],
       grad_fn=<AddBackward0>)

In [181]:
memory.shape

torch.Size([4, 1, 6])

In [182]:
def decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num, tgt_len, src_len, head_dim, memory_mask = None,need_weights = False):

    # (tgt_len, bsz, embed_dim)
    query_dec_mha = x_dec

    # (src_len, bsz, embed_dim)
    key_dec_mha, value_dec_mha = memory, memory


    tgt_len, bsz, embed_dim = query_dec_mha.shape


    W_dec_mha = state_dict["transformer.decoder.layers.{}.multihead_attn.in_proj_weight".format(layer_num)]
    b_dec_mha = state_dict["transformer.decoder.layers.{}.multihead_attn.in_proj_bias".format(layer_num)]


    ########## CHANGE FROM SELF ATTENTION ############

    Q_dec_mha,K_dec_mha,V_dec_mha = get_qkv(query_dec_mha, key_dec_mha, value_dec_mha ,W_dec_mha, b_dec_mha)


    #################################################

    K_dec_mha = K_dec_mha.unsqueeze(0)
    V_dec_mha = V_dec_mha.unsqueeze(0)
    Q_dec_mha = Q_dec_mha.unsqueeze(0)


    Q_dec_mha = Q_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    K_dec_mha = K_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    V_dec_mha = V_dec_mha.reshape(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)

    print("Q_dec_{} = ".format(layer_num))
    print(Q_dec_mha)
    print()

    print("K_dec_{} = ".format(layer_num))
    print(K_dec_mha)
    print()

    print("V_dec_{} = ".format(layer_num))
    print(V_dec_mha)
    print()


    attn_mask = memory_mask
    if attn_mask is not None:

        # Ensuring attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # attn_mask can be either (L,S) or (N*num_heads, L, S)
    # if attn_mask's shape is (1, L, S) we need to unsqueeze to (1, 1, L, S)
    # in order to match the input for SDPA of (N, num_heads, L, S)
    if attn_mask is not None:
        if attn_mask.size(0) == 1 and attn_mask.dim() == 3:
            attn_mask = attn_mask.unsqueeze(0)
        else:
            attn_mask = attn_mask.view(bsz, num_heads, -1, src_len)



    if need_weights is False:

        attn_output_dec_mha = atten_product_needs_wts_false(Q_dec_mha, V_dec_mha, K_dec_mha, bsz, head_dim, src_len, tgt_len, attn_mask)
        
        print("Cross attention in decoder_{}".format(layer_num))
        print(attn_output_dec_mha)
        print()

        op_dec_mha_1 = torch.matmul(attn_output_dec_mha, state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.bias".format(layer_num)]

        attn_dec_mha_output = op_dec_mha_1.view(tgt_len, bsz, attn_output_dec_mha.size(1))

        return attn_dec_mha_output, None
    
    else: 

    
        attn_dec_mha_output ,attn_wt_matrix_dec_mha = atten_product_needs_wts_true(Q_dec_mha, K_dec_mha, V_dec_mha, bsz, tgt_len, attn_mask)

        print("Decoder mha Attention output = ")
        print(attn_dec_mha_output)
        print()


        op_dec_mha = torch.matmul(attn_dec_mha_output, state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.multihead_attn.out_proj.bias".format(layer_num)]
        attn_dec_output_mha = op_dec_mha.view(tgt_len, bsz, attn_dec_mha_output.size(1))


        return attn_dec_output_mha , attn_wt_matrix_dec_mha

        


In [183]:
# need_weights = True

attn_dec_mha_output, attn_dec_mha_wts = decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num = 0, tgt_len = tgt_len, src_len = src_len, head_dim = head_dim, need_weights = need_weights)

Q_dec_0 = 
tensor([[[-1.5225, -1.3120, -1.9461],
         [-0.1991, -0.7104, -1.7021],
         [-1.0709, -0.5297, -1.1219],
         [-1.0709, -0.5297, -1.1219]],

        [[-0.8573, -0.2711,  0.2245],
         [-0.6236, -1.2013,  0.9954],
         [-1.0236, -0.4365, -0.0575],
         [-1.0236, -0.4365, -0.0575]]], grad_fn=<TransposeBackward0>)

K_dec_0 = 
tensor([[[ 0.7639,  0.3596,  0.6521],
         [ 1.4840,  0.8749, -0.3867],
         [ 0.7257,  0.4635,  0.0324],
         [ 1.0081,  0.8540, -0.2985]],

        [[-0.3901, -0.7216,  0.2131],
         [ 0.4722, -0.8736, -0.6255],
         [ 0.5111, -0.3893, -0.3831],
         [ 0.3892, -0.5258, -0.4995]]], grad_fn=<TransposeBackward0>)

V_dec_0 = 
tensor([[[ 0.5164,  0.2813,  0.6188],
         [ 0.5019, -0.7920,  0.7854],
         [ 0.7359,  0.4198,  0.5549],
         [ 0.6834, -0.3629,  0.4580]],

        [[ 0.3277,  0.5997,  0.3185],
         [ 0.9193, -0.4360,  0.3634],
         [ 0.6701,  0.4711,  0.9148],
         [ 1.2092,  0

In [184]:
def decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num):

    # (tgt_len, bsz, embed_dim)
    output_dec_2 = attn_dec_mha_output + x_dec

    linear_result_dec_2 = output_dec_2*state_dict["transformer.decoder.layers.{}.norm2.weight".format(layer_num)] + state_dict["transformer.decoder.layers.{}.norm2.bias".format(layer_num)]

    # Layer normalization from Torch's implementation 
    layernorm_dec_2 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_2.shape[2:])
    linear_op_dec_2 = layernorm_dec_2(linear_result_dec_2)

    x = linear_result_dec_2
    w = layernorm_dec_2.weight
    b = layernorm_dec_2.bias

    linear_result_dec_2f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_2f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_2f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_2 = (linear_result_dec_2f - mean) / (std + epsilon) * w + b

    print("norm2(x' + mha(x', mem)), \n where x' = Decoder_curr_layer norm1(x + sa(x))")
    print(normalized_result_dec_2)
    print("\n\n")

    x_dec2_norm = normalized_result_dec_2

    op_dec_1 = torch.matmul(x_dec2_norm, state_dict["transformer.decoder.layers.{}.linear1.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.linear1.bias".format(layer_num)]
    op_dec_1_relu = torch.nn.functional.relu(op_dec_1)
    op_dec_2 = torch.matmul(op_dec_1_relu, state_dict["transformer.decoder.layers.{}.linear2.weight".format(layer_num)].t()) + state_dict["transformer.decoder.layers.{}.linear2.bias".format(layer_num)]

    ff_dec = op_dec_2

    x_dec3_unorm = x_dec2_norm + ff_dec

    linear_result_dec_3 = x_dec3_unorm*state_dict["transformer.decoder.layers.0.norm3.weight"] + state_dict["transformer.decoder.layers.0.norm3.bias"]

    # Layer normalization from Torch's implementation 
    layernorm_dec_3 = torch.nn.LayerNorm(normalized_shape=linear_result_dec_3.shape[2:])
    linear_op_dec_3 = layernorm_dec_3(linear_result_dec_3)

    # From torch's implementation
    x = linear_result_dec_3

    # Obtained layer norm weights and biases (learnable)
    w = layernorm_dec_3.weight
    b = layernorm_dec_3.bias

    linear_result_dec_3f = w*x + b

    epsilon = 1e-5  
    mean = linear_result_dec_3f.mean(dim=-1, keepdim=True)
    std = linear_result_dec_3f.std(dim=-1, unbiased=False, keepdim=True)
    normalized_result_dec_3 = (linear_result_dec_3f - mean) / (std + epsilon) * w + b

    print("norm3(x'' + ff(x'')) \n where, x'' = Decoder_curr_layer norm2(x' + mha(x'))")
    print(normalized_result_dec_3)
    print()

    return normalized_result_dec_3

def feef_fwd_transformer(dec_output_final, state_dict):

    # (tgt_len, bsz, embed_dim) @ (vocab_size, embed_dim).T -> (tgt_len, bsz, vocab_size)
    final_op = dec_output_final@state_dict["fc.weight"].T + state_dict["fc.bias"]

    return final_op

In [185]:
state_dict["fc.weight"].shape

torch.Size([10, 6])

In [186]:
final_attn_op_decoder = decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num = 0)

norm2(x' + mha(x', mem)), 
 where x' = Decoder_curr_layer norm1(x + sa(x))
tensor([[[-1.6626, -0.2054,  1.1871,  0.8788, -0.8033,  0.6054]],

        [[-0.7633, -1.5386,  0.7136,  1.5919, -0.0597,  0.0562]],

        [[-2.1009,  0.1303,  1.0147, -0.0620,  0.4149,  0.6030]],

        [[-2.1009,  0.1303,  1.0147, -0.0620,  0.4149,  0.6030]]],
       grad_fn=<AddBackward0>)



norm3(x'' + ff(x'')) 
 where, x'' = Decoder_curr_layer norm2(x' + mha(x'))
tensor([[[-1.3175, -0.6332,  1.6384,  0.3114, -0.7350,  0.7358]],

        [[-0.2502, -1.9815,  0.8999,  1.0663,  0.2534,  0.0120]],

        [[-1.6847, -0.1621,  1.3100, -0.7599,  0.6732,  0.6234]],

        [[-1.6847, -0.1621,  1.3100, -0.7599,  0.6732,  0.6234]]],
       grad_fn=<AddBackward0>)



In [187]:
final_attn_op_decoder.shape

torch.Size([4, 1, 6])

In [188]:
# (tgt_len, bsz, vocab_dim)
final_op = feef_fwd_transformer(final_attn_op_decoder, state_dict)

In [189]:
final_op

tensor([[[-0.3931,  0.7839, -0.0701, -0.8229,  0.0118, -0.7191, -0.1468,
          -0.8998,  0.1445,  0.0056]],

        [[-0.4864,  1.2767,  0.2504, -1.1761, -0.0962, -0.3620, -0.3499,
          -0.1509, -0.5999, -0.1273]],

        [[-0.6542,  0.5709,  0.3987, -0.5503, -0.5313, -1.0139, -0.1259,
          -0.5488,  1.0464, -0.3315]],

        [[-0.6542,  0.5709,  0.3987, -0.5503, -0.5313, -1.0139, -0.1259,
          -0.5488,  1.0464, -0.3315]]], grad_fn=<AddBackward0>)

########################################################################

### Examaple :- 

Transformer with 3 encoders and 3 decoders

In [201]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return self.encoding[:, :x.size(1)].detach()

class TransformerModel(nn.Module):
    
    def __init__(self, src_vocab_size, tgt_vocab_size,max_seq_len, num_encoder_layers=1, num_decoder_layers=1, d_model=4, nhead=2):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0, max_len=max_seq_len)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=0
        )
        self.fc = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt):

        print("Source embeddings :- \n")
        print(self.src_embedding(src))
        print()

        print("Target embeddings :- \n")
        print(self.tgt_embedding(tgt))
        print()


        src = self.src_embedding(src) + self.positional_encoding(src)
        tgt = self.tgt_embedding(tgt) + self.positional_encoding(tgt)

        print("Positional encoded source embeddings:")
        print(src)
        print()

        print("Positional encoded target embeddings:")
        print(tgt)
        print()

        output = self.transformer(src, tgt)
        output = self.fc(output)
        
        return output
    

src_vocab_size = 10  # Source language vocabulary size
tgt_vocab_size = 10  # Target language vocabulary size
d_model = 4  # Dimension of the model
num_heads = 2
num_encoder_layers = 3
num_decoder_layers = 3

need_weights = False

src_mask = None
tgt_mask = None
memory_mask = None

max_seq_len = 4

d_model = 6


model = TransformerModel(src_vocab_size, tgt_vocab_size, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, d_model=d_model, max_seq_len = max_seq_len, nhead=num_heads)
# Source sentence in the source language
src_sentence = torch.tensor([[0], [1], [2], [3]])  # Source sequence

# Target sentence in the target language (translation of the source sentence)
tgt_sentence = torch.tensor([[1], [0], [3], [3]])  # Target sequence

# Forward pass
output = model(src_sentence, tgt_sentence)
print(output)


Source embeddings :- 

tensor([[[-0.5329,  0.4479, -1.5247, -0.6124,  0.9608, -0.5078]],

        [[ 0.7777,  0.7221, -0.2767, -1.7240, -0.0987,  0.7716]],

        [[-0.2460,  1.2069,  0.8297,  0.9564, -0.2230, -2.0615]],

        [[-0.6652, -0.4723,  0.9936, -0.3680,  1.2617,  0.3828]]],
       grad_fn=<EmbeddingBackward0>)

Target embeddings :- 

tensor([[[-0.3454,  0.9345, -0.6539,  1.9464,  2.3963, -1.9435]],

        [[ 0.6227,  1.4822,  3.2615, -0.7944, -0.7245, -1.0534]],

        [[ 0.8797, -0.3861,  0.2234, -0.2576, -1.4788, -1.3086]],

        [[ 0.8797, -0.3861,  0.2234, -0.2576, -1.4788, -1.3086]]],
       grad_fn=<EmbeddingBackward0>)

Positional encoded source embeddings:
tensor([[[-0.5329,  1.4479, -1.5247,  0.3876,  0.9608,  0.4922]],

        [[ 0.7777,  1.7221, -0.2767, -0.7240, -0.0987,  1.7716]],

        [[-0.2460,  2.2069,  0.8297,  1.9564, -0.2230, -1.0615]],

        [[-0.6652,  0.5277,  0.9936,  0.6320,  1.2617,  1.3828]]],
       grad_fn=<AddBackward0>)

Posi

/Users/sreevaatsav/.pyenv/versions/project_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [202]:
def get_all_intermediate_outputs(src_sentence, tgt_sentence,d_model, model, num_encoder_layers , num_decoder_layers):

    state_dict = model.state_dict()

    pe_src_embeds, pe_tgt_embeds = get_embedding_outputs(src_sentence=src_sentence,  tgt_sentence=tgt_sentence, state_dict=state_dict, max_seq_len=max_seq_len, d_model = d_model)
    print("###"*25)
    print("### Encoder Start ###")
    print()

    x_enc = pe_src_embeds

    for lno in range(num_encoder_layers):
        attn_enc_output, src_len, head_dim, attn_weights = encoder_block_attn_output(x_enc, state_dict, layer_num = lno, need_weights = False, embed_dim = d_model, num_heads= num_heads, src_mask=None)

        if lno == 0:
            tgt_len, bsz, embed_dim = x_enc.shape

        output_enc_final = encoder_block_post_attn_output(x_enc, attn_enc_output, state_dict, layer_num = lno , bsz = bsz, tgt_len = tgt_len)

        x_enc = output_enc_final

    
    print("### Encoder Done ###")
    print("\n\n\n")
    print("### Decoder Start ###")

    x_dec = pe_tgt_embeds
    memory = x_enc

    for lno in range(num_decoder_layers):

        self_attn_dec, dec_sa_wts = decoder_block_self_attn_output(x_dec, state_dict, layer_num = lno, need_weights = False, tgt_mask=None)
        x_dec = dec_post_self_attn(self_attn_dec, x_dec, state_dict, layer_num = lno)

        attn_dec_mha_output, attn_dec_mha_wts = decoder_block_cross_attn_output(x_dec, memory, state_dict, layer_num = lno, tgt_len = tgt_len, src_len = src_len, head_dim = head_dim, need_weights = False, memory_mask=None)
        final_op = decoder_block_post_attn_output(x_dec, attn_dec_mha_output, state_dict, layer_num = lno)

        print(pe_tgt_embeds.shape, final_op.shape)
        x_dec = final_op

    print("### Decoder Done ###")


    final_op = feef_fwd_transformer(final_op, state_dict)


    
    return final_op

    

In [203]:
d_model = 6
num_encoder_layers = 3
num_decoder_layers = 3

final_op = get_all_intermediate_outputs(src_sentence, tgt_sentence, model = model, num_encoder_layers = num_encoder_layers , num_decoder_layers = num_encoder_layers, d_model=d_model)

Source sentence embedding
Word index: 0, Embedding: tensor([-0.5329,  0.4479, -1.5247, -0.6124,  0.9608, -0.5078])
Word index: 1, Embedding: tensor([ 0.7777,  0.7221, -0.2767, -1.7240, -0.0987,  0.7716])
Word index: 2, Embedding: tensor([-0.2460,  1.2069,  0.8297,  0.9564, -0.2230, -2.0615])
Word index: 3, Embedding: tensor([-0.6652, -0.4723,  0.9936, -0.3680,  1.2617,  0.3828])

torch.Size([4, 1, 6])
Target sentence embedding
Word index: 1, Embedding: tensor([-0.3454,  0.9345, -0.6539,  1.9464,  2.3963, -1.9435])
Word index: 0, Embedding: tensor([ 0.6227,  1.4822,  3.2615, -0.7944, -0.7245, -1.0534])
Word index: 3, Embedding: tensor([ 0.8797, -0.3861,  0.2234, -0.2576, -1.4788, -1.3086])
Word index: 3, Embedding: tensor([ 0.8797, -0.3861,  0.2234, -0.2576, -1.4788, -1.3086])

PE of src :
tensor([[[0., 1., 0., 1., 0., 1.]]])

PE of tgt :
tensor([[[0., 1., 0., 1., 0., 1.]]])

PE source embeddings : 

tensor([[[-0.5329,  1.4479, -1.5247,  0.3876,  0.9608,  0.4922]],

        [[ 0.7777,  

In [208]:
output.shape, final_op.shape


(torch.Size([4, 1, 10]), torch.Size([4, 1, 10]))

In [209]:
output

tensor([[[ 0.2080, -0.2101, -0.3444, -0.4398,  0.0395,  0.8219, -0.1725,
           0.7168, -0.5166, -0.1150]],

        [[-0.5421,  0.0616, -0.9904,  0.5963,  0.2021,  1.1265,  0.1808,
           0.8478, -0.8467,  0.4492]],

        [[-0.8845, -0.1459, -1.2355,  0.2183,  0.2404,  0.4411, -0.5962,
           0.2322, -0.0240,  0.6122]],

        [[-0.8845, -0.1459, -1.2355,  0.2183,  0.2404,  0.4411, -0.5962,
           0.2322, -0.0240,  0.6122]]], grad_fn=<ViewBackward0>)

In [210]:
final_op

tensor([[[ 0.2080, -0.2101, -0.3444, -0.4398,  0.0395,  0.8219, -0.1725,
           0.7168, -0.5166, -0.1150]],

        [[-0.5421,  0.0616, -0.9904,  0.5963,  0.2021,  1.1265,  0.1808,
           0.8478, -0.8467,  0.4492]],

        [[-0.8845, -0.1459, -1.2354,  0.2183,  0.2404,  0.4411, -0.5962,
           0.2322, -0.0240,  0.6122]],

        [[-0.8845, -0.1459, -1.2354,  0.2183,  0.2404,  0.4411, -0.5962,
           0.2322, -0.0240,  0.6122]]], grad_fn=<AddBackward0>)